# Group weather data spatially and temporally

In [1]:
import pandas as pd
import os

In [2]:
# Loop through each year from 2016 to 2023
for curr_year in range(2016, 2023):

    # Define the base directory where the station data is located
    base_dir = "../data/processing/weather-stations/kelowna/"

    # Loop through each of the 36 fine areas
    for i in range(1, 37):
        # Read the CSV file for the current fine area
        filtered_df = pd.read_csv(base_dir + f"fine_area_{i}_stations.csv")

        # Define the columns of interest for filtering weather data
        my_filter = ['ONE_DAY_PRECIPITATION', 'ONE_DAY_RAIN', 'MIN_TEMP', 'ONE_DAY_SNOW', 'time', 'MAX_TEMP']

        # Loop through each week of the year (1 to 52)
        for week in range(1, 53):
            weekly_dataframes = []  # Initialize a list to store weekly dataframes

            # Iterate through each row in the filtered station DataFrame
            for index, row in filtered_df.iterrows():
                network_name = row['Network Name']  # Get the network name for the station
                native_id = row['Native ID']  # Get the native ID for the station

                # Construct the file path for the weather data for the current week, network, and station
                file_path = f'../data/raw/weather/{curr_year}/week_{week}/{network_name}/{native_id}.csv'

                # Check if the weather data file exists for the current station and week
                if os.path.exists(file_path):
                    # Read the weather data CSV file and append it to the weekly dataframes list
                    temp_df = pd.read_csv(file_path)
                    weekly_dataframes.append(temp_df)
                    
            # If any dataframes were collected for the current week, combine them
            if weekly_dataframes:
                # Concatenate all the dataframes into a single DataFrame
                combined_df = pd.concat(weekly_dataframes, ignore_index=True, axis=0, join='outer')
                
                # Check if the output directory for the current year and week exists, if not, create it
                if not os.path.exists(f'../data/processing/final-weather/{curr_year}/week_{week}'):
                    os.makedirs(f'../data/processing/final-weather/{curr_year}/week_{week}')
                
                # Define the file path to save the combined DataFrame as a CSV
                combined_csv_path = f'../data/processing/final-weather/{curr_year}/week_{week}/fine_area_{i}_weather.csv'
                
                # Save the combined DataFrame to the defined CSV path
                combined_df.to_csv(combined_csv_path, index=False, header=True)